# Auto Correlation with TMC, CSP.LMC, and CBF in the Mid PSI
##### Last Updated: 2024-04-08

This demo will show the basic operation of a 4-receptor 200MHz correlation driven from TMC to CSP.LMC and CBF. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

### Pre-requisites

This notebook has the following assumptions and pre-requisites:
* A deployment that includes TMC, CSP.LMC, and CBF is running. For example, one in a namespace `NS` that was launched from the [ska-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline.
* Taranta should be enabled in the deployment and the CSP.LMC Monitoring Dashboard can be used to monitor the relevant CSP.LMC and CBF devices.
* A virtual environment is being used. This notebook was made with Python 3.10 in mind.
* All requirements are installed via [poetry]().
* The talon boards `TARGET_TALONS` are powered off.
* The Engineering Console scripts are run using kubectl commands. Once the EC Device Servers are fully function, this notebook file can be updated to incorporate them instead.
* This notebook assumes there are 4-receptors and 1 FSP. Once 4 FSPs are available, it can be updated to incorporate that.

## Execution Steps

### Initial Parameter Set Up

The parameters below (except for `TANGO_HOST`) may need to be updated to match your deployment, environment, and paths. The config files are located within the ska-mid-jupyter-notebooks repository in a subdirectory called `data`.

NOTE: There are two hardware config files for the Mid PSI environment. `hw_config_psi.yaml` is used for Talons 1-4 and `hw_config_psi_swap.yaml` is used for Talons 5-8 in the Mid PSI. If USE_SWAP_HW_CONFIG is set to `True`, the HW config file for Talons 5-8 will be used, otherwise the standard HW config file is used.

In [ ]:
import os
from time import sleep

print("Setting up parameters")

NS = "ci-ska-mid-psi-1245030042-maecst"  # Update namespace accordingly
BITE_MAC_ADDRESS = "08:c0:eb:9d:47:78"

# TANGO HOST set up
CLUSTER_DOMAIN = "cluster.local"
TANGO_DB = "databaseds-tango-base"
TANGO_HOST = "10.164.10.119:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

TARGET_TALON_LIST = [1, 2, 3, 4]
DELIM = ","
TARGET_TALONS = DELIM.join(map(str, TARGET_TALON_LIST))

RECEPTORS = ["SKA001", "SKA036", "SKA063", "SKA100"]

# Config files set up
DATA_DIR = "../../data"
SCAN_FILE = f"{DATA_DIR}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
USE_SWAP_HW_CONFIG = False  # Update to True if testing on any of talon boards 5-8
if USE_SWAP_HW_CONFIG:
    print("Using swap HW config file ")
    HW_CONFIG_FILE = f"{CBF_CONFIGS}/hw_config/hw_config_swap_psi.yaml"
else:
    print("Using standard HW config file")
    HW_CONFIG_FILE = f"{CBF_CONFIGS}/hw_config/hw_config_psi.yaml"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"
SLIM_CONFIG_FILE = f"{CBF_CONFIGS}/slim_config/fs_slim_4vcc_1fsp.yaml"

### Set Up and Configure Device Proxies 

In [ ]:
import json

from pytango import DeviceProxy

print("Setting up the device proxies")

# TMC proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

sleep(2)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

### Load the Dish Vcc Config / Init Sys Params

In [ ]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = f.read()

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(dish_config_json)

sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

### Turn the Telescope On

In [ ]:
print("Running the TelescopeOn command")
tmc_central_node.TelescopeOn()

print("Monitor the kubectl logs for the CBF Controller by running in a shell:")
print("kubectl logs -n $NS ds-cbfcontroller-controller-0 -f")

# kubectl logs -n $NS ds-cbfcontroller-controller-0 -f

In [ ]:
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

### Monitor Visibilities Pod

These steps should be done in a separate terminal to monitor the packets when replaying the BITE data command a bit below.

In [ ]:
# kubectl apply -f viz_pod.yaml
# kubectl exec -ti sdn-dynamic-vis-1  -- bash
# apt update && apt install -y iproute2 tcpdump
# ip a

# Update the IP in the Configure Scan to be the IP of the inet interface of the visibilities pod

# tcpdump -i net1

### Assign Resources

In [ ]:
print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS


print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))

sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

### Configure Scan

In [ ]:
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

tmc_subarray.Configure(json.dumps(configure_scan_json))

sleep(5)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

In [ ]:
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

### Replay the BITE Data using Engineering Console Bite

In [ ]:
print("Replay the BITE Data")

!kubectl exec -ti -n {NS} ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards={TARGET_TALONS}

print("DONE")

### Scan

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning")

with open(SCAN_FILE, encoding="utf-8") as f:
    scan_json = f.read()

print(f"\nscan_json file contents: \n{scan_json}")

tmc_csp_subarray.Scan(scan_json)

sleep(2)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

### Clean Up


In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

tmc_csp_subarray.EndScan()

sleep(2)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

tmc_csp_subarray.End()

sleep(2)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

tmc_csp_subarray.ReleaseAllResources()

sleep(2)
print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

### Telescope Off

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

sleep(5)
print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")